Plan for this Notebook:
- Data Exploration of Raw Polls Data set
- Festure eng - Playing with Spacy model to see what we can extract from "comment" - paid by political candidate or organization (lots of news orgs or candidates)
- 

In [6]:
#imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import spacy

In [7]:
nlp = spacy.load("en_core_web_sm")

In [2]:
polls = pd.read_csv('../data/raw-polls.csv')

polls.head()

,poll_id,question_id,race_id,year,race,location,type_simple,type_detail,pollster,pollster_rating_id,...,margin_poll,electiondate,cand1_actual,cand2_actual,margin_actual,error,bias,rightcall,advancedplusminus,comment
0,26013,87909,1455,1998,1998_Gov-G_NY,NY,Gov-G,Gov-G,Blum & Weprin Associates,32,...,-31.0,11/3/1998,33.16,54.32,-21.15,9.85,-9.85,1.0,1.461561,for New York Daily News | WABC-TV (New York)
1,26255,87926,1456,1998,1998_Gov-G_OH,OH,Gov-G,Gov-G,University of Cincinnati (Ohio Poll),346,...,-15.0,11/3/1998,44.69,50.05,-5.36,9.64,-9.64,1.0,4.285789,NaN
2,26026,31266,1736,1998,1998_Sen-G_NV,NV,Sen-G,Sen-G,FM3 Research,91,...,5.0,11/3/1998,47.86,47.77,0.09,4.91,4.91,1.0,-1.023784,for unspecified Democratic sponsor
3,26013,31253,1738,1998,1998_Sen-G_NY,NY,Sen-G,Sen-G,Blum & Weprin Associates,32,...,-8.0,11/3/1998,54.62,44.08,10.54,18.54,-18.54,0.0,10.064104,for New York Daily News | WABC-TV (New York)
4,63632,117103,1738,1998,1998_Sen-G_NY,NY,Sen-G,Sen-G,Garin-Hart-Yang Research Group,113,...,4.0,11/3/1998,54.62,44.08,10.54,6.54,-6.54,1.0,-1.972391,for Charles E. Schumer


In [3]:
polls['electiondate'].str.split('/').apply(lambda x: x[2]).astype(int) == polls['year']

0         True
1         True
2         True
3         True
4         True
         ...  
10771    False
10772    False
10773    False
10774    False
10775    False
Length: 10776, dtype: bool

In [4]:
polls.shape

(10776, 33)

In [22]:
polls["comment"].unique()

array(['for New York Daily News | WABC-TV (New York)', nan,
       'for unspecified Democratic sponsor', 'for Charles E. Schumer',
       'for Richard Stallings',
       'sample size unavailable; estimated at 600 as a default',
       'for Tom Vilsack', 'for Richard A. Hill', 'for Robert C. Hayes',
       'for CNN | Time', 'among registered voters',
       'for Gary Anthony Hofmeister', 'for The Washington Post',
       'among all adults', 'for unspecified Republican sponsor',
       'for Chicago Sun-Times', 'for The Salt Lake Tribune',
       'average of multiple versions or turnout models listed in poll',
       'for Russ Feingold',
       'for AP; sample size unavailable; estimated at 600 as a default',
       'for USA Today | CNN',
       'sample size unavailable; estimated at 600 as a default; among registered voters',
       'for Wall Street Journal; among registered voters', 'for NBC News',
       'for Bob Inglis; average of multiple versions or turnout models listed in poll',
 

In [9]:
def org_flag(list):
    for tup in list:
        if tup[0] == 'ORG':
            return 1
    return 0

def org_finder(text):
    doc = nlp(text)

    ents = [(ent.label_, ent.text) for ent in doc.ents]

    return org_flag(ents)

In [10]:
polls['org'] = polls['comment'].apply(lambda x: org_finder(x) if x is not np.nan else 0)

In [12]:
polls.head()

,poll_id,question_id,race_id,year,race,location,type_simple,type_detail,pollster,pollster_rating_id,...,electiondate,cand1_actual,cand2_actual,margin_actual,error,bias,rightcall,advancedplusminus,comment,org
0,26013,87909,1455,1998,1998_Gov-G_NY,NY,Gov-G,Gov-G,Blum & Weprin Associates,32,...,11/3/1998,33.16,54.32,-21.15,9.85,-9.85,1.0,1.461561,for New York Daily News | WABC-TV (New York),1
1,26255,87926,1456,1998,1998_Gov-G_OH,OH,Gov-G,Gov-G,University of Cincinnati (Ohio Poll),346,...,11/3/1998,44.69,50.05,-5.36,9.64,-9.64,1.0,4.285789,NaN,0
2,26026,31266,1736,1998,1998_Sen-G_NV,NV,Sen-G,Sen-G,FM3 Research,91,...,11/3/1998,47.86,47.77,0.09,4.91,4.91,1.0,-1.023784,for unspecified Democratic sponsor,0
3,26013,31253,1738,1998,1998_Sen-G_NY,NY,Sen-G,Sen-G,Blum & Weprin Associates,32,...,11/3/1998,54.62,44.08,10.54,18.54,-18.54,0.0,10.064104,for New York Daily News | WABC-TV (New York),1
4,63632,117103,1738,1998,1998_Sen-G_NY,NY,Sen-G,Sen-G,Garin-Hart-Yang Research Group,113,...,11/3/1998,54.62,44.08,10.54,6.54,-6.54,1.0,-1.972391,for Charles E. Schumer,0


In [13]:
def person_flag(list):
    for tup in list:
        if tup[0] == 'PERSON':
            return 1
    return 0

def person_finder(text):
    doc = nlp(text)

    ents = [(ent.label_, ent.text) for ent in doc.ents]

    return person_flag(ents)

In [17]:
polls['person'] = polls['comment'].apply(lambda x: person_finder(x) if x is not np.nan else 0)

In [20]:
polls.head(100)

,poll_id,question_id,race_id,year,race,location,type_simple,type_detail,pollster,pollster_rating_id,...,cand1_actual,cand2_actual,margin_actual,error,bias,rightcall,advancedplusminus,comment,org,person
0,26013,87909,1455,1998,1998_Gov-G_NY,NY,Gov-G,Gov-G,Blum & Weprin Associates,32,...,33.16,54.32,-21.15,9.85,-9.85,1.0,1.461561,for New York Daily News | WABC-TV (New York),1,0
1,26255,87926,1456,1998,1998_Gov-G_OH,OH,Gov-G,Gov-G,University of Cincinnati (Ohio Poll),346,...,44.69,50.05,-5.36,9.64,-9.64,1.0,4.285789,NaN,0,0
2,26026,31266,1736,1998,1998_Sen-G_NV,NV,Sen-G,Sen-G,FM3 Research,91,...,47.86,47.77,0.09,4.91,4.91,1.0,-1.023784,for unspecified Democratic sponsor,0,0
3,26013,31253,1738,1998,1998_Sen-G_NY,NY,Sen-G,Sen-G,Blum & Weprin Associates,32,...,54.62,44.08,10.54,18.54,-18.54,0.0,10.064104,for New York Daily News | WABC-TV (New York),1,0
4,63632,117103,1738,1998,1998_Sen-G_NY,NY,Sen-G,Sen-G,Garin-Hart-Yang Research Group,113,...,54.62,44.08,10.54,6.54,-6.54,1.0,-1.972391,for Charles E. Schumer,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,54186,87721,1433,1998,1998_Gov-G_AL,AL,Gov-G,Gov-G,University of South Alabama,361,...,57.81,42.19,15.62,1.62,-1.62,1.0,-4.457890,NaN,0,0
96,54335,87871,1449,1998,1998_Gov-G_MI,MI,Gov-G,Gov-G,Mitchell Research & Communications,213,...,37.78,62.20,-24.43,7.57,-7.57,1.0,1.613796,NaN,0,0
97,54413,87949,1460,1998,1998_Gov-G_RI,RI,Gov-G,Gov-G,KRC Research,160,...,42.13,50.97,-8.84,13.84,13.84,0.0,7.099101,NaN,0,0
98,26017,31257,1722,1998,1998_Sen-G_CO,CO,Sen-G,Sen-G,Ciruli Associates,51,...,35.02,62.49,-27.47,3.53,-3.53,1.0,-2.511903,NaN,0,0
